<a href="https://colab.research.google.com/github/kiplangatkorir/MetaLearning/blob/main/stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/conedata.csv')
data.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,ID,test_type,basin_valley,Depth (m),qc (MPa),fs (kPa),u2 (kPa),Vs (m/s),qt (MPa),Rf (%),...,Fr (%),Bq (-),U2 (-),SBT (-),SBTn (-),Mod. SBTn (-),n,Ic (-),Ic SBT (-),Ib (-)
0,0,CPT,Zell basin,0.02,1.15,1.0,0,0,1.77,0.06,...,0.06,0,0,0,0,0,-0.12,2.56,2.23,25.61
1,0,CPT,Zell basin,0.04,2.85,1.0,0,0,2.99,0.03,...,0.03,0,0,0,0,0,0.33,1.23,2.02,256.60
2,0,CPT,Zell basin,0.06,4.80,1.0,0,0,4.76,0.06,...,0.06,0,0,0,0,0,0.28,1.16,1.80,264.66
3,0,CPT,Zell basin,0.08,6.40,6.0,0,0,6.12,0.07,...,0.07,0,0,0,0,0,0.27,1.12,1.69,275.17
4,0,CPT,Zell basin,0.10,6.90,6.0,0,0,6.91,0.10,...,0.10,0,0,0,0,0,0.27,1.11,1.65,267.92


In [105]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [109]:
# Features
data = pd.read_csv('/content/drive/MyDrive/conedata.csv')
# Features
X = data[['Depth (m)', 'qc (MPa)', 'fs (kPa)', 'u2 (kPa)', 'Vs (m/s)']]

# Target variables
y1 = data['SBT (-)']
y2 = data['Ic SBT (-)']

In [110]:
data.head()
data.info()
data.columns
(data.shape)
(data.describe())
(data.isnull().sum())
(data.dtypes)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2876 entries, 0 to 2875
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             2876 non-null   int64  
 1   test_type      2876 non-null   object 
 2   basin_valley   2876 non-null   object 
 3   Depth (m)      2876 non-null   float64
 4   qc (MPa)       2876 non-null   float64
 5   fs (kPa)       2876 non-null   float64
 6   u2 (kPa)       2876 non-null   int64  
 7   Vs (m/s)       2876 non-null   int64  
 8   qt (MPa)       2876 non-null   float64
 9   Rf (%)         2876 non-null   float64
 10  γ (kN/m³)      2876 non-null   int64  
 11  σ,v (kPa)      2876 non-null   float64
 12  u0 (kPa)       2876 non-null   float64
 13  σ',v (kPa)     2876 non-null   float64
 14  Qt (-)         2876 non-null   float64
 15  Qtn (-)        2876 non-null   float64
 16  Fr (%)         2876 non-null   float64
 17  Bq (-)         2876 non-null   int64  
 18  U2 (-)  

ID                 int64
test_type         object
basin_valley      object
Depth (m)        float64
qc (MPa)         float64
fs (kPa)         float64
u2 (kPa)           int64
Vs (m/s)           int64
qt (MPa)         float64
Rf (%)           float64
γ (kN/m³)          int64
σ,v (kPa)        float64
u0 (kPa)         float64
σ',v (kPa)       float64
Qt (-)           float64
Qtn (-)          float64
Fr (%)           float64
Bq (-)             int64
U2 (-)             int64
SBT (-)            int64
SBTn (-)           int64
Mod. SBTn (-)      int64
n                float64
Ic (-)           float64
Ic SBT (-)       float64
Ib (-)           float64
dtype: object

In [113]:
# Split data into train and test sets
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)
_, _, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)


In [125]:
# Define a list of base models
base_models = [
    RandomForestRegressor(),
    XGBRegressor(),
]

In [126]:
# Train base models and make predictions on the test set
predictions = []
for model in base_models:
    model.fit(X_train, y1_train)
    y1_pred = model.predict(X_test)
    predictions.append(y1_pred.reshape(-1, 1))

In [128]:
# Train meta-model (combiner algorithm) using base models predictions
meta_X = np.hstack(predictions)
meta_y = y1_test.values

meta_model = LinearRegression()
meta_model.fit(meta_X, meta_y)


LinearRegression()

In [129]:
# Make final predictions using the meta-model
final_predictions = meta_model.predict(meta_X)

In [132]:
# Evaluate the performance of the stacking model
mse = mean_squared_error(meta_y, final_predictions)
r2 = r2_score(meta_y, final_predictions)
print(f"Stacking Model MSE: {mse:.4f}")
print(f"Stacking Model R-squared: {r2:.4f}")

Stacking Model MSE: 0.0490
Stacking Model R-squared: 0.9750
